# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [16]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [60]:
import numpy as np
import random


def basic_local_search(graph):
    
    
    def random_divide(opt_part1, opt_part2): 
        # Строим изначальное произвольное разбиение.
        # Возвращаем вес этого разбиения.
        size = len(graph[0])//2
        segment = len(graph[0])
        while len(opt_part1) <= size:
            new = random.randint(1, segment)
            opt_part1.add(new)
            
        for i in range(0, segment):
            if (i + 1) not in opt_part1:
                opt_part2.add(i + 1)
                
        return opt_part1, opt_part2, get_weight(opt_part1, opt_part2)

    
    def get_weight(p1, p2):
        # Ищем вес текущего разбиения.
        weight = 0
        for v1 in p1:
            for v2 in p2:
                if (v1, v2) in graph[1]:
                    weight += 1
        return weight
    
    
    def try_changing():
        # Попытка улучшить текущий результат.
        # Возвращаем вес оптимального разбиения в 1-окрестности.
        part1 = opt_part1.copy() # Копии разбиения, которые будут меняться
        part2 = opt_part2.copy()
        box1 = set()
        box2 = set()
        cur_best_part1 = part1.copy() # Запоминаем текущее разбиение как лучшее
        cur_best_part2 = part2.copy()
        cur_weight = get_weight(part1, part2) # Текущий вес
        best_weight = get_weight(part1, part2) # Лучший вес
        for first in opt_part1: # Перебираем все возможные пары вершин и меняем их местами
            for second in opt_part2:
                box1 = {first}
                box2 = {second}
                part1 = part1 - box1
                part1 = part1 | box2
                part2 = part2 - box2
                part2 = part2 | box1 
                new_weight = get_weight(part1, part2)
                if(best_weight > new_weight): # Результат улучшился
                    cur_best_part1 = part1.copy()
                    cur_best_part2 = part2.copy()
                    best_weight = new_weight
                part1 = opt_part1.copy() # Возвращаемся на исходную позицию
                part1 = opt_part2.copy()
        return cur_best_part1, cur_best_part2, best_weight
    
    size = len(graph[0]) 
    opt_part1 = set()
    opt_part2 = set()
    opt_part1, opt_part2, current_weight = random_divide(opt_part1, opt_part2)
    
    fails = 0
    result = current_weight
    current_weight += 1
    while result < current_weight: # Повторяем, пока результат улучшается
        current_weight = result
        opt_part1, opt_part2, result = try_changing()
    return opt_part1

In [61]:
print(basic_local_search(read_col_file('myciel3.col')))

{8, 10, 3, 4, 7}


In [54]:
print(basic_local_search(read_col_file('myciel4.col')))

{4, 6, 8, 9, 10, 11, 13, 14, 15, 16, 22}


In [55]:
print(basic_local_search(read_col_file('myciel5.col')))

{2, 4, 5, 6, 9, 11, 13, 15, 18, 19, 20, 21, 23, 25, 27, 31, 33, 34, 41, 42, 44, 45, 47}


In [56]:
print(basic_local_search(read_col_file('myciel6.col')))

{2, 7, 8, 9, 10, 11, 12, 14, 16, 18, 19, 20, 22, 26, 28, 29, 33, 34, 36, 38, 39, 42, 43, 45, 46, 47, 48, 49, 51, 53, 55, 57, 58, 61, 62, 65, 73, 75, 76, 77, 79, 81, 82, 91, 92, 93, 94}


In [57]:
print(basic_local_search(read_col_file('myciel7.col')))

{2, 3, 4, 6, 7, 9, 15, 18, 19, 20, 23, 27, 29, 31, 32, 33, 35, 36, 38, 39, 41, 43, 45, 46, 50, 51, 54, 55, 56, 59, 60, 61, 64, 65, 67, 70, 73, 75, 77, 78, 79, 80, 84, 88, 89, 90, 92, 94, 95, 99, 101, 102, 108, 109, 111, 112, 113, 114, 119, 120, 121, 123, 128, 129, 130, 132, 134, 136, 137, 139, 140, 141, 142, 145, 146, 148, 149, 150, 151, 154, 155, 158, 164, 168, 171, 172, 174, 175, 176, 179, 181, 184, 185, 186, 187}
